In [13]:
import os
import yaml

os.chdir('Z:\OMOP\omop_etl')
print(os.getcwd())

import pandas as pd
from omop_etl.datastore import DataStore
# from .omop_etl.stage import STAGE

from pydma.utils import find

Z:\OMOP\omop_etl


In [14]:
omop = DataStore('config.yml')

CONFIG = 'omop_etl/etl_config.yml'
with open(CONFIG) as f:
    yml = yaml.safe_load(f)

ALIASES = yml['aliases']
STAGE = yml['stage']

In [4]:
stage_t = omop.list_tables(in_schema=['stage'])
stage_t['Count'] = stage_t.Table.apply(lambda t: omop.row_count(t, schema='stage'))

In [5]:
stage_t_dict = stage_t[['Table','Count']].to_dict()
# stage_t_dict

In [6]:
stage_t_dict['Count'][7]

5172282

In [19]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [19]:
loaded = omop.config_param['load']

result = []
for t in loaded.keys():
    if t not in ['provider','care_site','location']:
        if loaded[t]:
            for part in loaded[t]:
                stg_name = STAGE[t][part]
                idx = find_key(stage_t_dict['Table'], stg_name)
                count = stage_t_dict['Count'][idx]
                result.append([t, part, count])
        else:
            stg_name = STAGE[t]
            idx = find_key(stage_t_dict['Table'], stg_name)
            count = stage_t_dict['Count'][idx]
            result.append([t, None, count])

table_counts = pd.DataFrame(result, columns=['Table', 'Part', 'Count'])
table_counts.head(50)

,Table,Part,Count
0,person,None,261222
1,death,None,6452
2,condition_occurrence,None,74725451
3,procedure_occurrence,icd,396109
4,procedure_occurrence,cpt,29049589
5,drug_exposure,order,28348416
6,drug_exposure,admin,51170789
7,measurement,bp,21078360
8,measurement,bp_artline,6664673
9,measurement,bp_noninvasive,5173404


In [24]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff

,Table,Count,RC_Load,RC_DeId
0,condition_occurrence,74725451,73464094,20867090
1,death,6452,6446,5579
2,drug_exposure,79519205,78815931,34173541
3,measurement,315736168,360396063,275651510
4,observation,80449088,80169785,40503251
5,person,261222,261227,242388
6,procedure_occurrence,29445698,29201416,4790264
7,visit_occurrence,26752594,26663500,9669131


In [22]:
# t_sbs = stage_t[find(['MEASUREMENT','DRUG','CONDITION','PROCEDURE','OBSERVATION'], stage_t.Table)].Table.tolist()

In [30]:
count_diff = table_counts.groupby('Table').sum().reset_index()
count_diff['RC_Load'] = count_diff.Table.apply(lambda t: omop.row_count(t))
count_diff['RC_DeId'] = count_diff.Table.apply(lambda t: omop.row_count(t, schema='hipaa'))
count_diff

,Table,Count,RC_Load,RC_DeId
0,condition_occurrence,74725451,61154887,61154102
1,death,6452,6452,6452
2,drug_exposure,79519205,79394081,79393862
3,measurement,315736168,382511603,382511603
4,observation,80449088,99003172,99001854
5,person,261222,261227,261220
6,procedure_occurrence,29445698,13389500,13389310
7,visit_occurrence,26752594,26663500,26663500
